# Catalog Wrangling Exercise  
*:auth: Nate Stevens (PNSN)*

In this exercise we'll use ObsPy and ObsPlus to create a highly translatable earthquake catalog from events we located through analyses.  

The notebook will demonstrate some tools for getting our (meta)data into two well-defined schema:  
 - the ANSS schema (https://ncedc.org/db/Documents/NewSchemas/PI/v1.6.4/PI.1.6.4/index.htm)
 - the QuakeML schema (https://quake.ethz.ch/quakeml)

Finally, we'll make an EventBank that lets us save, update, and query our event catalog in a systematic manner.

In [1]:
import os
from pathlib import Path
from glob import glob
import pandas as pd
from obspy import read_events, UTCDateTime
import obsplus
from obspy.geodetics import gps2dist_azimuth, kilometer2degrees
from obspy.clients.fdsn import Client


In [2]:
# TODO: Make sure this points at wherever you saved your HypoDD outputs
ROOT = Path.cwd()
DATA = ROOT/'data'
CATD = ROOT/'catalog_files'
os.makedirs(str(CATD), exist_ok=True)
print(f'The data directory is registered as {DATA}')

The data directory is registered as /Users/nates/Code/GitHub/2025_ML_TSC/notebooks/Nate/data


In [3]:
# Load the HypoDD output into an ObsPy `Catalog` object
flist = glob(str(DATA/'*.pha'))
for _e, _f in enumerate(flist):
    if _e == 0:
        cat = read_events(_f)
    else:
        cat += read_events(_f)


display(cat)

1161 Event(s) in Catalog:
2022-12-20T00:57:25.304000Z | +40.645, -124.300 | 0.46 None
2022-12-20T01:21:51.303000Z | +39.771, -122.541 | 1.94 None
...
2022-12-20T23:58:40.340000Z | +40.633, -123.979 | 1.13 None
2022-12-20T23:59:34.450000Z | +40.624, -124.005 | 0.62 None
To see all events call 'print(CatalogObject.__str__(print_all=True))'

In [4]:
# Use ObsPlus to show a DataFrame representation of events (takes a little time)
df_events = cat.to_df()

### Compare this table the ORIGIN table in the ANSS schema

https://ncedc.org/db/Documents/NewSchemas/PI/v1.6.4/PI.1.6.4/Content/Tbl_388b5374f81611d6bcce00c04f794c81.htm

#### ...and look at all those empty fields, just waiting for you to populate them!



In [5]:
# Display our new table (conveniently formatted in nearly ANSS EVENT table format!)
display(df_events)

,time,latitude,longitude,depth,magnitude,event_description,associated_phase_count,azimuthal_gap,event_id,horizontal_uncertainty,...,standard_error,used_phase_count,station_count,vertical_uncertainty,updated,author,agency_id,creation_time,version,stations
0,2022-12-20 00:57:25.304,40.6448,-124.3003,8156.0,0.459,,28.0,NaN,smi:local/event/1,NaN,...,0.503,28.0,17.0,166.0,NaT,,,NaT,,"1023.NP, 1582.NP, B045.PB, B046.PB, B047.PB, B..."
1,2022-12-20 01:21:51.303,39.7714,-122.5413,22506.0,1.941,,12.0,NaN,smi:local/event/2,NaN,...,0.398,12.0,8.0,569.0,NaT,,,NaT,,"GHGB.NC, GROB.NC, GSR.NC, GTSB.BK, GVA.NC, KBN..."
2,2022-12-20 02:44:52.382,40.4189,-124.2338,23295.0,0.012,,15.0,NaN,smi:local/event/3,NaN,...,0.319,15.0,10.0,385.0,NaT,,,NaT,,"B045.PB, B046.PB, B047.PB, B049.PB, B932.PB, B..."
3,2022-12-20 03:06:59.875,40.3154,-124.3610,20037.0,0.914,,35.0,NaN,smi:local/event/4,NaN,...,0.333,35.0,20.0,193.0,NaT,,,NaT,,"1586.NP, 89688.CE, B045.PB, B046.PB, B047.PB, ..."
4,2022-12-20 03:18:27.700,40.3648,-124.2772,28226.0,0.909,,36.0,NaN,smi:local/event/5,NaN,...,0.477,36.0,21.0,219.0,NaT,,,NaT,,"89688.CE, B045.PB, B046.PB, B047.PB, B049.PB, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,2022-12-20 23:54:08.504,40.5428,-123.9475,19852.0,0.568,,24.0,NaN,smi:local/event/1157,NaN,...,0.453,24.0,14.0,209.0,NaT,,,NaT,,"B046.PB, B049.PB, B932.PB, B935.PB, BJES.BK, D..."
1157,2022-12-20 23:55:52.393,40.5971,-124.3612,20997.0,0.752,,17.0,NaN,smi:local/event/1158,NaN,...,0.408,17.0,10.0,605.0,NaT,,,NaT,,"B046.PB, B047.PB, B932.PB, B935.PB, KCR.NC, KM..."
1158,2022-12-20 23:56:38.054,40.5076,-124.2864,19549.0,0.735,,19.0,NaN,smi:local/event/1159,NaN,...,0.501,19.0,13.0,347.0,NaT,,,NaT,,"B046.PB, B047.PB, B049.PB, B932.PB, B935.PB, K..."
1159,2022-12-20 23:58:40.340,40.6331,-123.9790,20756.0,1.130,,32.0,NaN,smi:local/event/1160,NaN,...,0.190,32.0,17.0,232.0,NaT,,,NaT,,"1581.NP, B046.PB, B047.PB, B049.PB, B932.PB, B..."


## Althought the ObsPlus documentation is sometimes sparese on examples, their coding is quite good!
Let's turn all of our picks into a dataframe

In [6]:
# Turns out the *.pha I/O for ObsPy has a little bug, so we need to apply a small correction to assign network and station codes to the correct fields
try:
    df_picks = cat.arrivals_to_df()
except:
    for event in cat.events:
        for pick in event.picks:
            sn = pick.waveform_id.station_code
            pick.waveform_id.station_code=sn.split('.')[0]
            pick.waveform_id.network_code=sn.split('.')[1]
    df_picks = cat.arrivals_to_df()

### Compare this to the ARRIVAL and ASSOCARO tables in the ANSS schema

#### ARRIVAL
https://ncedc.org/db/Documents/NewSchemas/PI/v1.6.4/PI.1.6.4/Content/Tbl_388b5400f81611d6bcce00c04f794c81.htm

#### ASSOCARO (Association of Arrivals and Origins)
https://ncedc.org/db/Documents/NewSchemas/PI/v1.6.4/PI.1.6.4/Content/Tbl_388b542ef81611d6bcce00c04f794c81.htm

In [7]:
display(df_picks)

,resource_id,seed_id,pick_id,phase,time_correction,azimuth,distance,takeoff_angle,time_residual,horizontal_slowness_residual,...,earth_model_id,creation_time,author,agency_id,network,station,location,channel,origin_id,origin_time
0,smi:local/619b5da6-7f8c-4463-a37d-7e0087ae60dc,NP.1582..Z,smi:local/559ab90d-b0f6-4fe6-b20b-72fc12c99ae5,P,NaN,NaN,NaN,NaN,NaN,NaN,...,,NaT,,,NP,1582,,Z,smi:local/origin/1,2022-12-20 00:57:25.304
1,smi:local/d75dba0e-0d71-4282-8db2-76f94946d430,NC.KCT..Z,smi:local/b364b7b7-9b5a-4ce7-ab7f-cb5887f1ecc6,P,NaN,NaN,NaN,NaN,NaN,NaN,...,,NaT,,,NC,KCT,,Z,smi:local/origin/1,2022-12-20 00:57:25.304
2,smi:local/f783e6db-6e06-432d-8055-d009383bfb4a,BK.KNEE..Z,smi:local/f5760581-397b-4e11-8385-6d810f146844,P,NaN,NaN,NaN,NaN,NaN,NaN,...,,NaT,,,BK,KNEE,,Z,smi:local/origin/1,2022-12-20 00:57:25.304
3,smi:local/9f6bb218-1b57-48ec-af72-32de910efde6,PB.B046..Z,smi:local/b9b06fb3-787f-45ad-89d9-7125dc852ac9,P,NaN,NaN,NaN,NaN,NaN,NaN,...,,NaT,,,PB,B046,,Z,smi:local/origin/1,2022-12-20 00:57:25.304
4,smi:local/9c60b375-0679-4688-a0ce-d7a632128468,PB.B932..Z,smi:local/2fd3357b-2959-4fca-8502-e6db63713e60,P,NaN,NaN,NaN,NaN,NaN,NaN,...,,NaT,,,PB,B932,,Z,smi:local/origin/1,2022-12-20 00:57:25.304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39378,smi:local/c4a50b28-71d0-4e6a-abe8-7ccba748cc33,PB.B932..N,smi:local/034cbadd-610b-475a-8190-66a84bcf2a02,S,NaN,NaN,NaN,NaN,NaN,NaN,...,,NaT,,,PB,B932,,N,smi:local/origin/1161,2022-12-20 23:59:34.450
39379,smi:local/bfcd35c9-6dc6-494e-be0f-0a6427ba28ab,PB.B049..N,smi:local/b2ed1ad0-380a-41c7-a5cf-a73027fec467,S,NaN,NaN,NaN,NaN,NaN,NaN,...,,NaT,,,PB,B049,,N,smi:local/origin/1161,2022-12-20 23:59:34.450
39380,smi:local/ee5e4525-ae2e-433c-85a6-373c8d93a10d,NC.KMR..N,smi:local/63505811-ec83-4142-8814-66592a759b3d,S,NaN,NaN,NaN,NaN,NaN,NaN,...,,NaT,,,NC,KMR,,N,smi:local/origin/1161,2022-12-20 23:59:34.450
39381,smi:local/0048ef42-f837-4de2-b907-a5f8343e080a,BK.PRDS..N,smi:local/600476a0-5621-4a8b-b779-7c58f9e1daa0,S,NaN,NaN,NaN,NaN,NaN,NaN,...,,NaT,,,BK,PRDS,,N,smi:local/origin/1161,2022-12-20 23:59:34.450


# Now that we've populated an ObsPy Catalog object, we can write into a bunch of different formats

QuakeML is a well-described, extensible schema for seismic event (meta)data exchange  
https://quake.ethz.ch/quakeml 

- ObsPy saves `Catalog` objects in this format as default  

**BUT** before you go saving everything as one big QuakeML file, be warned that they can get large and slow to read from disk.  

You can find one of my past sins against easily accessible (albiet well formatted) data here:  
https://zenodo.org/records/8393876


Instead, let's put our catalog into a tidy directory structure with a client interface!  
Another place where `ObsPlus` shines!  

In [8]:
# Initialize an event bank
ebank = obsplus.EventBank(base_path=CATD/'EventBank',
                          path_structure='{year}/{month}/{day}/{hour}',
                          name_structure='{event_id_end}',
                          format='quakeml')

In [9]:
# Add events to eventbank, and take a look at your file directory!
ebank.put_events(cat)

  0% (0 of 1161) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--
  4% (50 of 1161) |                      | Elapsed Time: 0:00:00 ETA:   0:00:18
  8% (100 of 1161) |#                    | Elapsed Time: 0:00:01 ETA:   0:00:18
 12% (150 of 1161) |##                   | Elapsed Time: 0:00:02 ETA:   0:00:17
 17% (200 of 1161) |###                  | Elapsed Time: 0:00:03 ETA:   0:00:17
 21% (250 of 1161) |####                 | Elapsed Time: 0:00:04 ETA:   0:00:15
 25% (300 of 1161) |#####                | Elapsed Time: 0:00:05 ETA:   0:00:14
 30% (350 of 1161) |######               | Elapsed Time: 0:00:06 ETA:   0:00:13
 34% (400 of 1161) |#######              | Elapsed Time: 0:00:06 ETA:   0:00:12
 38% (450 of 1161) |########             | Elapsed Time: 0:00:08 ETA:   0:00:14
 43% (500 of 1161) |#########            | Elapsed Time: 0:00:08 ETA:   0:00:12
 47% (550 of 1161) |#########            | Elapsed Time: 0:00:09 ETA:   0:00:11
 51% (600 of 1161) |##########          

EventBank(base_path=/Users/nates/Code/GitHub/2025_ML_TSC/notebooks/Nate/catalog_files/EventBank)

In [10]:
# Get a summary of events in your event bank
display(ebank.read_index())

,time,latitude,longitude,depth,magnitude,event_description,associated_phase_count,azimuthal_gap,event_id,horizontal_uncertainty,...,standard_error,used_phase_count,station_count,vertical_uncertainty,updated,author,agency_id,creation_time,version,path
0,2022-12-20 00:57:25.304,40.6448,-124.3003,8156.0,0.459,,28.0,NaN,smi:local/event/1,NaN,...,0.503,28.0,17.0,166.0,2025-05-09 22:32:20.094966784,,,NaT,,2022/12/20/00/1.xml
1,2022-12-20 01:21:51.303,39.7714,-122.5413,22506.0,1.941,,12.0,NaN,smi:local/event/2,NaN,...,0.398,12.0,8.0,569.0,2025-05-09 22:32:20.095594496,,,NaT,,2022/12/20/01/2.xml
2,2022-12-20 02:44:52.382,40.4189,-124.2338,23295.0,0.012,,15.0,NaN,smi:local/event/3,NaN,...,0.319,15.0,10.0,385.0,2025-05-09 22:32:20.096281600,,,NaT,,2022/12/20/02/3.xml
3,2022-12-20 03:06:59.875,40.3154,-124.3610,20037.0,0.914,,35.0,NaN,smi:local/event/4,NaN,...,0.333,35.0,20.0,193.0,2025-05-09 22:32:20.097510656,,,NaT,,2022/12/20/03/4.xml
4,2022-12-20 03:18:27.700,40.3648,-124.2772,28226.0,0.909,,36.0,NaN,smi:local/event/5,NaN,...,0.477,36.0,21.0,219.0,2025-05-09 22:32:20.098684672,,,NaT,,2022/12/20/03/5.xml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,2022-12-20 23:54:08.504,40.5428,-123.9475,19852.0,0.568,,24.0,NaN,smi:local/event/1157,NaN,...,0.453,24.0,14.0,209.0,2025-05-09 22:32:21.389415680,,,NaT,,2022/12/20/23/1157.xml
1157,2022-12-20 23:55:52.393,40.5971,-124.3612,20997.0,0.752,,17.0,NaN,smi:local/event/1158,NaN,...,0.408,17.0,10.0,605.0,2025-05-09 22:32:21.390056704,,,NaT,,2022/12/20/23/1158.xml
1158,2022-12-20 23:56:38.054,40.5076,-124.2864,19549.0,0.735,,19.0,NaN,smi:local/event/1159,NaN,...,0.501,19.0,13.0,347.0,2025-05-09 22:32:21.390759936,,,NaT,,2022/12/20/23/1159.xml
1159,2022-12-20 23:58:40.340,40.6331,-123.9790,20756.0,1.130,,32.0,NaN,smi:local/event/1160,NaN,...,0.190,32.0,17.0,232.0,2025-05-09 22:32:21.392038400,,,NaT,,2022/12/20/23/1160.xml


### Now let's prove to ourself that this EventBank thingy is persistent

In [11]:
# DELETE the EventBank Object in our session
del ebank
try:
    display(ebank)
except NameError:
    print('ebank object does not exist, as expected')

ebank object does not exist, as expected


In [12]:
# Re-initialize connection to the EventBank
ebank = obsplus.EventBank(base_path=CATD/'EventBank')
display(ebank)
# Note that the `path_structure` or `name_structure` key-word arguments we defined are saved!
print('Our Event Bank values')
display(ebank.path_structure)
display(ebank.name_structure)
print('Default values')
print('{year}/{month}/{day}')
print('{time}_{event_id_short}')

EventBank(base_path=/Users/nates/Code/GitHub/2025_ML_TSC/notebooks/Nate/catalog_files/EventBank)

Our Event Bank values


'{year}/{month}/{day}/{hour}'

'{event_id_end}'

Default values
{year}/{month}/{day}
{time}_{event_id_short}


In [13]:
# Query a subset of events
# Read the index (a pandas DataFrame)
df_index = ebank.read_index()
# Subset by origin times
_df_index = df_index[(df_index.time >= pd.Timestamp('2022-12-20T20:00:00')) & (df_index.time <= pd.Timestamp('2022-12-20T21:40:00'))]
# Get events from your event bank
cat = ebank.get_events(event_id=_df_index.event_id.values)

display(cat)


109 Event(s) in Catalog:
2022-12-20T21:24:28.925000Z | +40.600, -124.060 | 1.19 None
2022-12-20T21:25:56.938000Z | +40.518, -124.295 | 0.96 None
...
2022-12-20T21:22:52.282000Z | +40.565, -124.066 | 1.50 None
2022-12-20T21:24:05.763000Z | +40.586, -124.069 | 0.51 None
To see all events call 'print(CatalogObject.__str__(print_all=True))'

### Let's modify some event metadata and submit it to our EventBank
In this case, let's add distances and back-azimuths to associated phases  

In [14]:
# Let's populate some source-receiver geometry information
client = Client('IRIS')
nets = ','.join(list(df_picks.network.unique()))
stas = ','.join(list(df_picks.station.unique()))
inv = client.get_stations(network=nets, station=stas, level='channel',starttime=UTCDateTime('20221220'), endtime=UTCDateTime('20221221'))

# Use ObsPlus added methods to convert the inventory into a dataframe
df_stations = inv.to_df()

display(df_stations)

,network,station,location,channel,seed_id,latitude,longitude,elevation,depth,azimuth,dip,sample_rate,start_date,end_date
0,BK,BJES,00,BHE,BK.BJES.00.BHE,40.543620,-123.879960,1047.9,2.7,90.0,0.0,40.000000,2019-08-22,2024-05-22 20:00:00
1,BK,BJES,00,BHN,BK.BJES.00.BHN,40.543620,-123.879960,1047.9,2.7,0.0,0.0,40.000000,2019-08-22,2024-05-22 20:00:00
2,BK,BJES,00,BHZ,BK.BJES.00.BHZ,40.543620,-123.879960,1047.9,2.7,0.0,-90.0,40.000000,2019-08-22,2024-05-22 20:00:00
3,BK,BJES,00,HHE,BK.BJES.00.HHE,40.543620,-123.879960,1047.9,2.7,90.0,0.0,100.000000,2019-08-22,2024-05-22 20:00:00
4,BK,BJES,00,HHN,BK.BJES.00.HHN,40.543620,-123.879960,1047.9,2.7,0.0,0.0,100.000000,2019-08-22,2024-05-22 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,PB,B935,T8,RCB,PB.B935.T8.RCB,40.478699,-123.573196,696.7,158.2,203.1,0.0,0.000278,2008-10-28,NaT
1515,PB,B935,T8,RCC,PB.B935.T8.RCC,40.478699,-123.573196,696.7,158.2,143.1,0.0,0.000278,2008-10-28,NaT
1516,PB,B935,T8,RCD,PB.B935.T8.RCD,40.478699,-123.573196,696.7,158.2,113.1,0.0,0.000278,2008-10-28,NaT
1517,PB,B935,TS,RDO,PB.B935.TS.RDO,40.478699,-123.573196,696.7,0.0,0.0,0.0,0.000556,2008-10-28,NaT


In [15]:
# Add the maximum azimuthal gap to each origin
# Here's a starting point:

# Iterate across events
origin_gaps = []
for event in cat.events:
    # Iterate across origins
    for origin in event.origins:
        olon = origin.longitude
        olat = origin.latitude
        # Iterate across associated arrivals
        bazs = set([])
        for arrival in origin.arrivals:
            # Get pick observations
            pick = arrival.pick_id.get_referred_object()
            # Get station location
            network = pick.waveform_id.network_code
            station = pick.waveform_id.station_code
            _df_sta = df_stations[(df_stations.network==network) & (df_stations.station==station)][['station','network','latitude','longitude']]
            try:
                slon = _df_sta.longitude.values[0]
                slat = _df_sta.latitude.values[0]
            except:
                continue
            # Get distances
            dist_m, seaz, esaz = gps2dist_azimuth(slat, slon, olat, olon)
            # Convert distance to degrees
            arrival.distance = kilometer2degrees(dist_m*1e-3)
            # Assign back-azimuth
            arrival.azimuth = esaz

## A task for the HACK-A-THON, get azimuthal gaps into your EventBank

#             bazs.add(esaz)

        
#         # Calculate gaps
#         bazs = list(bazs)
#         bazs.sort()
#         gaps = [bazs[_e+1] - bazs[_e] for _e in range(len(bazs)-1)] + [360 - bazs[-1] + bazs[0]]
#         # Get maximum azimuthal gap
#         maxgap = max(gaps)
#         # associate with resourceID
#         origin_gaps.append([origin.resource_id.id, maxgap])

# # An exercise for users to incorporate 'gap' values into their preferred schema
# display(pd.DataFrame(origin_gaps, columns=['resource_id','gap']))

In [16]:
# Show that the geometry data stuck
display(cat.events[0].origins[0].arrivals)

[Arrival
	 resource_id: ResourceIdentifier(id="smi:local/c01bfa56-ee9d-4c51-8f58-5725834e8120")
	     pick_id: ResourceIdentifier(id="smi:local/2eb2f58e-ff97-4a15-9009-78c114b0eb2b")
	       phase: 'P'
	 time_weight: 0.87,
 Arrival
	 resource_id: ResourceIdentifier(id="smi:local/c07fa430-b75c-4128-9b70-00381c1c7015")
	     pick_id: ResourceIdentifier(id="smi:local/da47ce49-9423-463d-bdb9-53429da1e6dc")
	       phase: 'P'
	     azimuth: 112.15165160393657
	    distance: 0.14842161918964963
	 time_weight: 0.57,
 Arrival
	 resource_id: ResourceIdentifier(id="smi:local/17ad1a34-ec58-4f56-8e7f-6d1b9da15a76")
	     pick_id: ResourceIdentifier(id="smi:local/66dddcf7-b089-4bb2-9df9-29a6c6841a65")
	       phase: 'P'
	     azimuth: 46.37878777472903
	    distance: 0.1671958835350023
	 time_weight: 0.66,
 Arrival
	 resource_id: ResourceIdentifier(id="smi:local/6ee49fc1-fdd3-4549-8a48-6e68b7427708")
	     pick_id: ResourceIdentifier(id="smi:local/a75fcd15-f4bd-4274-bda8-b79eb7537897")
	       phas

In [17]:
# Submit the catalog back to the event bank to update
ebank.put_events(cat)

  0% (0 of 110) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
 45% (50 of 110) |##########             | Elapsed Time: 0:00:00 ETA:   0:00:00
 90% (100 of 110) |####################  | Elapsed Time: 0:00:01 ETA:   0:00:00
100% (110 of 110) |######################| Elapsed Time: 0:00:01 Time:  0:00:01


EventBank(base_path=/Users/nates/Code/GitHub/2025_ML_TSC/notebooks/Nate/catalog_files/EventBank)

In [18]:
# Delete `cat` and re-load to prove to ourselves that the geometry information was saved
del cat
try:
    display(cat)
except NameError:
    print('cat does not exist, as expected')

cat does not exist, as expected


In [19]:
# Re-load the sub-catalog
cat = ebank.get_events(event_id = _df_index.event_id)
display(cat)

109 Event(s) in Catalog:
2022-12-20T21:24:28.925000Z | +40.600, -124.060 | 1.19 None
2022-12-20T21:25:56.938000Z | +40.518, -124.295 | 0.96 None
...
2022-12-20T21:22:52.282000Z | +40.565, -124.066 | 1.50 None
2022-12-20T21:24:05.763000Z | +40.586, -124.069 | 0.51 None
To see all events call 'print(CatalogObject.__str__(print_all=True))'

In [20]:
# View that the geometry data persist on events we modified
display(cat.events[0].origins[0].arrivals)

[Arrival
	 resource_id: ResourceIdentifier(id="smi:local/c01bfa56-ee9d-4c51-8f58-5725834e8120")
	     pick_id: ResourceIdentifier(id="smi:local/2eb2f58e-ff97-4a15-9009-78c114b0eb2b")
	       phase: 'P'
	 time_weight: 0.87,
 Arrival
	 resource_id: ResourceIdentifier(id="smi:local/c07fa430-b75c-4128-9b70-00381c1c7015")
	     pick_id: ResourceIdentifier(id="smi:local/da47ce49-9423-463d-bdb9-53429da1e6dc")
	       phase: 'P'
	     azimuth: 112.15165160393657
	    distance: 0.14842161918964963
	 time_weight: 0.57,
 Arrival
	 resource_id: ResourceIdentifier(id="smi:local/17ad1a34-ec58-4f56-8e7f-6d1b9da15a76")
	     pick_id: ResourceIdentifier(id="smi:local/66dddcf7-b089-4bb2-9df9-29a6c6841a65")
	       phase: 'P'
	     azimuth: 46.37878777472903
	    distance: 0.1671958835350023
	 time_weight: 0.66,
 Arrival
	 resource_id: ResourceIdentifier(id="smi:local/6ee49fc1-fdd3-4549-8a48-6e68b7427708")
	     pick_id: ResourceIdentifier(id="smi:local/a75fcd15-f4bd-4274-bda8-b79eb7537897")
	       phas

In [21]:
# Load all the events and check an unmodified event
cat = ebank.get_events()


In [22]:
# Display catalog (should see everything)
display(cat)
# Display arrivals for the the first event's origin, which we did not add back azimuths or distances to.
display(cat.events[0].origins[0].arrivals)

1161 Event(s) in Catalog:
2022-12-20T00:57:25.304000Z | +40.645, -124.300 | 0.46 None
2022-12-20T01:21:51.303000Z | +39.771, -122.541 | 1.94 None
...
2022-12-20T21:22:52.282000Z | +40.565, -124.066 | 1.50 None
2022-12-20T21:24:05.763000Z | +40.586, -124.069 | 0.51 None
To see all events call 'print(CatalogObject.__str__(print_all=True))'

[Arrival
	 resource_id: ResourceIdentifier(id="smi:local/619b5da6-7f8c-4463-a37d-7e0087ae60dc")
	     pick_id: ResourceIdentifier(id="smi:local/559ab90d-b0f6-4fe6-b20b-72fc12c99ae5")
	       phase: 'P'
	 time_weight: 0.71,
 Arrival
	 resource_id: ResourceIdentifier(id="smi:local/d75dba0e-0d71-4282-8db2-76f94946d430")
	     pick_id: ResourceIdentifier(id="smi:local/b364b7b7-9b5a-4ce7-ab7f-cb5887f1ecc6")
	       phase: 'P'
	 time_weight: 0.78,
 Arrival
	 resource_id: ResourceIdentifier(id="smi:local/f783e6db-6e06-432d-8055-d009383bfb4a")
	     pick_id: ResourceIdentifier(id="smi:local/f5760581-397b-4e11-8385-6d810f146844")
	       phase: 'P'
	 time_weight: 0.71,
 Arrival
	 resource_id: ResourceIdentifier(id="smi:local/9f6bb218-1b57-48ec-af72-32de910efde6")
	     pick_id: ResourceIdentifier(id="smi:local/b9b06fb3-787f-45ad-89d9-7125dc852ac9")
	       phase: 'P'
	 time_weight: 0.83,
 Arrival
	 resource_id: ResourceIdentifier(id="smi:local/9c60b375-0679-4688-a0ce-d7a632128468")
	     pick_i